In [1]:
import pandas as pd

In [ ]:
pages = pd.read_csv('simple_english_wiki_pages.csv')
links = pd.read_csv('simple_english_wiki_pagelinks.csv')
# на git загружать не буду, предполагаем, что нужно указать правильные пути, если хотите запустить и проверить

In [3]:
pages.head()

,page_id,page_title
0,1,April
1,2,August
2,6,Art
3,8,A
4,9,Air


In [4]:
links.info

<bound method DataFrame.info of          pl_from            pl_title   pl_to
0              1                1320   39168
1           7560                1320   39168
2           7873                1320   39168
3           8393                1320   39168
4           8719                1320   39168
...          ...                 ...     ...
6724923   836310  Anglo-Manipuri_War  836099
6724924   836311  Anglo-Manipuri_War  836099
6724925   836313    Battle_of_Imphal  836107
6724926   836316     Alfredo_Borrero  834696
6724927   836341    Nokia_6260_slide  833009

[6724928 rows x 3 columns]>

In [5]:
links = links.drop(columns=['pl_title']).drop_duplicates()
pl_from = list(links.pl_from)
pl_to = list(links.pl_to)

In [6]:
adjacency = {}

for _to, _from in zip(pl_to, pl_from):
    if _from in adjacency:
        adjacency[_from].append(_to)
    else:
        adjacency[_from] = [_to]

In [24]:
mapping = {}
reverse_mapping = {}

for i, name in zip(pages.page_id, pages.page_title):
    mapping[name] = i
    reverse_mapping[i] = name



In [8]:
# Task 1
from collections import deque

def bfs(adj, s, t):
    if s == t:
        return 0
    q = deque([s])
    dist = {s: 0}
    while q:
        u = q.popleft()
        for v in adj.get(u, []):
            if v not in dist:
                dist[v] = dist[u] + 1
                if v == t:
                    return dist[v]
                q.append(v)
    return -1

In [18]:
name1 = 'Analytics'
name2 = 'Algorithm'

i_from, i_to = mapping[name1], mapping[name2]

print(f"Кратчайшее расстояние между {name1} и {name2} равно {bfs(adjacency, i_from, i_to)}")

Кратчайшее расстояние между Analytics и Algorithm равно 4


In [19]:
# Task 2

def bfs_path(adj, s, t):
    if s == t:
        return [s]
    q = deque([s])
    p = {s: None}
    while q:
        u = q.popleft()
        for v in adj.get(u, []):
            if v not in p:
                p[v] = u
                if v == t:
                    path = []
                    while v is not None:
                        path.append(v)
                        v = p[v]
                    return path[::-1]
                q.append(v)
    return []

In [28]:
print(f"Статья на предыдущем пути, ссылающаяся на {name2}: {reverse_mapping[bfs_path(adjacency, i_from, i_to)[-2]]}")

Статья на предыдущем пути, ссылающаяся на Algorithm: Computer_science


In [ ]:
# Task 3

k = max(list(x for x in map(len, pages.page_title)))

INF = 10**18

def zero_k_bfs(adj, rev, s, t, k=k):
    if s == t:
        return 0
    maxd = len(adj)
    buckets = [deque() for _ in range(maxd + 1)]
    dist = {}
    dist[s] = 0
    buckets[0].append(s)
    cur = 0
    while cur <= maxd:
        if not buckets[cur]:
            cur += 1
            continue
        u = buckets[cur].popleft()
        if dist.get(u, INF) != cur:
            continue
        for v in adj.get(u, []):
            w = len(rev.get(v, []))
            nd = cur + w
            if nd < dist.get(v, INF):
                dist[v] = nd
                if nd <= maxd:
                    buckets[nd].append(v)
        if t in dist:
            return dist[t]
    return -1


In [34]:
print(f"Кратчайшее расстояние с учетом длины слова, на которое идет ссылка: {zero_k_bfs(adjacency, reverse_mapping, i_from, i_to)}")

Кратчайшее расстояние с учетом длины слова, на которое идет ссылка: 29


In [39]:
# Task 4

INF = 10**18

def zero_k_bfs_path(adj, rev, s, t, k=k):
    if s == t:
        return [s]
    maxd = len(adj)
    buckets = [deque() for _ in range(maxd + 1)]
    dist = {s: 0}
    parent = {s: None}
    buckets[0].append(s)
    cur = 0
    while cur <= maxd:
        if not buckets[cur]:
            cur += 1
            continue
        u = buckets[cur].popleft()
        if dist.get(u, INF) != cur:
            continue
        if u == t:
            path = []
            v = t
            while v is not None:
                path.append(v)
                v = parent[v]
            return path[::-1]
        for v in adj.get(u, []):
            w = len(rev.get(v, []))
            if w > k:
                w = k
            nd = cur + w
            if nd < dist.get(v, INF) and nd <= maxd:
                dist[v] = nd
                parent[v] = u
                buckets[nd].append(v)
    return []


In [40]:
print(f"Предпоследняя ссылка в предыдущем пути: {reverse_mapping[zero_k_bfs_path(adjacency, reverse_mapping, i_from, i_to)[-2]]}")

Предпоследняя ссылка в предыдущем пути: Logic
